In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import torch
from transformers import BertConfig
from fastformer import FastformerEncoder

In [4]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
bert = BertModel.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')

In [6]:
from data_utils import *
import pickle 

data_path = 'data/MINDsmall_train/'

build_news_id_map(data_path=data_path)
with open(data_path + 'news_id_map.pkl', 'rb') as f:
    news_id_map = pickle.load(f)
build_user_id_map(data_path=data_path)
with open(data_path + 'user_id_map.pkl', 'rb') as f:
    user_id_map = pickle.load(f)
build_news_token(news_id_map, data_path)
news_token = np.load(data_path + 'news_token.npy')

Tokenizing: 100%|██████████| 51283/51283 [01:06<00:00, 766.73it/s] 


In [7]:
bert_features = precompute_bert_features(bert, news_token, 'cuda')
torch.save(bert_features, data_path+'bert_features.pt')

100%|██████████| 802/802 [00:31<00:00, 25.42it/s]


In [ ]:
news_token

In [8]:
edge_index = build_edge_index(behavior_path=data_path+'behaviors.tsv', news_id_map=news_id_map, save_path=data_path+'edge_index.pt')

100%|██████████| 153727/153727 [00:03<00:00, 41034.63it/s]


In [9]:
build_behaviors(data_path + 'behaviors.tsv', user_id_map, news_id_map, data_path+'behaviors.pkl', 32)

156965it [00:04, 33124.33it/s]


In [ ]:
with open(data_path + 'behaviors.pkl', 'rb') as f:
    behaviors_data = pickle.load(f)

In [ ]:
from data import MINDDataset
from data_utils import collate_fn

In [ ]:
mind_data = MINDDataset(data_path + 'behaviors.pkl')

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(mind_data, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [ ]:
batch = next(iter(dataloader))

In [ ]:
print(batch['user_idx'].shape)
print(batch['history'].shape)
print(batch['candidates'].shape)
print(batch['seed_nodes'].shape)